# Pay Model Results Dashboard

In [1]:
%matplotlib inline

In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from datetime import date
from sqlalchemy import create_engine

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = [8, 6]

In [59]:
# Get the data for analysis
engine = create_engine("postgresql://Joachim:@localhost/tm470")
conn = engine.connect()
model_results_df = pd.read_sql_table('model_results_v', conn, index_col='id')
pay_options_df = pd.read_sql_table('pay_options_v', conn, index_col='id')
options_data_df = pd.read_sql_table('pay_options_data', conn, index_col='id')
pay_ranges_df = pd.read_sql_table('pay_ranges_v', conn, index_col='payband')
conn.close()

In [44]:
## Create separate DataFrames
consolidated_df = model_results_df[model_results_df['pay_option_type']=='Consolidated']
consolidated_df = consolidated_df.drop(['bonus_costs','bonus_percent'], axis=1)

bonus_df = model_results_df[model_results_df['pay_option_type']=='Non-consolidated']
bonus_df = bonus_df.drop(['consolidated_costs', 'consolidated_percent', 'uplift_costs', 'uplift_percent'], axis=1)

In [45]:
consolidated_df

,pay_option_name,pay_option_type,pay_period,model_params,consolidated_costs,consolidated_percent,uplift_costs,uplift_percent,paybill,runs,timestamp
id,,,,,,,,,,,
20,Pay award 16/17 - £1k (consolidated),Consolidated,2016/17,"{'breakpoint': {'1': 80000.0, '2': 110000.0, '...",136130.42,0.99,10419.0,0.08,13811459.0,500,2017-08-25 08:37:41.552833


In [46]:
options_data_df.pivot(index='payband_id', columns='key', values='value')

key,bonus,breakpoint,pay_award_tier1,pay_award_tier2,special_award,special_cases
payband_id,,,,,,
1,10750.0,80000.0,1000.0,250.0,2000.0,10.0
2,10750.0,110000.0,1000.0,250.0,2000.0,5.0
3,10750.0,135000.0,1000.0,250.0,NaN,NaN


# Pay ranges

In [60]:
pay_ranges_df

,min,max,period_name
payband,,,
1,64000,117800,2016/17
2,87000,162500,2016/17
3,106000,208100,2016/17


## Cost of uplift to new minima

In [136]:
print('£{:,}'.format(int(consolidated_df['uplift_costs'].values[0])))

£10,419


In [138]:
print('{:,}% of the pay bill'.format(consolidated_df['uplift_percent'].values[0]))

0.08% of the pay bill


# Pay option summary

## Consolidated

In [145]:
cons_option = pay_options_df[pay_options_df['type']=='Consolidated']
cons_option

,name,type,period_name,cutoff_date
id,,,,
1,Pay award 16/17 - £1k (consolidated),Consolidated,2016/17,2017-01-01


### Option Parameters

In [154]:
options_data_df[options_data_df['pay_option_id']==cons_option.index[0]].pivot(
    index='payband_id', columns='key', values='value')

key,breakpoint,pay_award_tier1,pay_award_tier2,special_award,special_cases
payband_id,,,,,
1,80000.0,1000.0,250.0,2000.0,10.0
2,110000.0,1000.0,250.0,2000.0,5.0
3,135000.0,1000.0,250.0,NaN,NaN
